In [1]:
import pandas as pd

In [2]:
rentals = pd.read_csv("dvd_rentals.rental.csv")
inventory = pd.read_csv("dvd_rentals.inventory.csv")
film = pd.read_csv("dvd_rentals.film.csv")
actor = pd.read_csv("dvd_rentals.actor.csv")
film_category = pd.read_csv("dvd_rentals.film_category.csv")
category = pd.read_csv("dvd_rentals.category.csv")
film_actor = pd.read_csv("dvd_rentals.film_actor.csv")

In [3]:
# we need to decide which sort of join to use 
# to that purpose we need to ask ourselves couple of questions
# How many records exist per inventory_id value in rental or inventory tables?
# How many overlapping and missing unique foreign key values are there between the two tables?
# hypotheses 1 - The number of unique inventory_id records will be equal in both dvd_rentals.rental
# and dvd_rentals.inventory tables

print(rentals["inventory_id"].nunique(), inventory["inventory_id"].nunique()) # hypotheses one is wrong
print() # just to get a new line

# it's possible that one inventory item doesn't get rented out, maybe its lost.
# hypotheses 2 - There will be a multiple records per unique inventory_id in the dvd_rentals.rental table

groupren = rentals["inventory_id"].value_counts().sort_values(ascending = False).reset_index().groupby("inventory_id")
print(groupren["index"].count()) # hypotheses 2 is correct
print()

# hypotheses 3 - There will be multiple inventory_id records per unique film_id value in the dvd_rentals.inventory table
invengrp = inventory["film_id"].value_counts().sort_values(ascending = False).reset_index().groupby("film_id")
print(invengrp["index"].count()) # hypotheses 3 is correct

4580 4581

inventory_id
1       4
2    1126
3    1151
4    1160
5    1139
Name: index, dtype: int64

film_id
2    133
3    131
4    183
5    136
6    187
7    116
8     72
Name: index, dtype: int64


In [4]:
# how to check if all the inventory id's in the inventory table exist on the rentals table
print(rentals.inventory_id.isin(inventory["inventory_id"]).value_counts())
print()

# all the inventory id's in the rentals table exist on the inventory table.
# how to check if all the inventory id's in the rentals table exist on the inventory table
print(inventory.inventory_id.isin(rentals["inventory_id"]).value_counts())
print()

# one inventory id from the inventory table does not exist on the rentals table (could be lost or a bad film no one wants\
# to watch)
# lets inspect it further 
print(inventory[~inventory.inventory_id.isin(rentals["inventory_id"])])
print()

# are there any duplicates inventory id's in the inventory table
print(inventory.inventory_id.duplicated().sum())
# no duplicate values, perfect fit for a left join to the rentals table

True    16044
Name: inventory_id, dtype: int64

True     4580
False       1
Name: inventory_id, dtype: int64

   inventory_id  film_id  store_id               last_update
4             5        1         2  2006-02-15T05:09:17.000Z

0


In [5]:
# Let’s perform the checklist steps for just the dvd_rentals.inventory and dvd_rentals.film tables.
rentals_by_customer = rentals[["customer_id","inventory_id"]].sort_values("customer_id")
rentals_by_customer = rentals_by_customer.merge(inventory[["inventory_id","film_id"]],\
                                                how = "left",left_on = "inventory_id", right_on ="inventory_id")

# how many unique foreign keys are on both the total movie table and the film table
print(film["film_id"].nunique(), rentals_by_customer["film_id"].nunique())
print()

# how many of the film id's in the total movies table exist on the film table
print(rentals_by_customer.film_id.isin(film["film_id"]).value_counts())
print()
# all of them exist

# are all film ids on the film table unique?
groupfilm = film["film_id"].value_counts().sort_values(ascending = False).reset_index().groupby("film_id")
print(groupfilm["index"].count()) # yes
print()

# safe to use a left join but we'll stick with danny's example and conpare both inner and left
rentals_by_customer_left = rentals_by_customer.merge(film[["film_id","title"]], how = "left", left_on = "film_id",\
                                                     right_on = "film_id") 
rentals_by_customer_inner = rentals_by_customer.merge(film[["film_id","title"]], how = "inner", left_on = "film_id",\
                                                      right_on = "film_id") 

captions = ["left join", "inner join"]
row_count = [len(rentals_by_customer_left), len(rentals_by_customer_inner)]
num_unique = [rentals_by_customer_left["film_id"].nunique(), rentals_by_customer_inner["film_id"].nunique()]
columns = ["join method","row_count","num_unique"]

complist = list(zip(captions, row_count, num_unique))

comp_df = pd.DataFrame(complist, columns = columns)
comp_df

1000 958

True    16044
Name: film_id, dtype: int64

film_id
1    1000
Name: index, dtype: int64



,join method,row_count,num_unique
0,left join,16044,958
1,inner join,16044,958


In [6]:
# so by now the new joined df looks like this
rentals_by_customer_left

,customer_id,inventory_id,film_id,title
0,1,797,174,CONFIDENTIAL INTERVIEW
1,1,726,159,CLOSER BANG
2,1,2639,579,MINDS TRUMAN
3,1,108,22,AMISTAD MIDSUMMER
4,1,2785,611,MUSKETEERS WAIT
...,...,...,...,...
16039,599,914,203,DAISY MENAGERIE
16040,599,2419,527,LOLA AGENT
16041,599,2033,443,HURRICANE AFFAIR
16042,599,4048,881,TEMPLE ATTRACTION


In [7]:
# joins 3 - film category
# how many unique foreign keys are on both the rentals_by_customer table and the film category table
print(film_category["film_id"].nunique(), rentals_by_customer_left["film_id"].nunique())
print()

# how many of the film id's in the rentals_by_customer table exist on the film category table
print(rentals_by_customer_left.film_id.isin(film_category["film_id"]).value_counts())
print()
# all of them exist

# are all film ids on the film table unique?
groupfilm = film_category["film_id"].value_counts().sort_values(ascending = False).reset_index().groupby("film_id")
print(groupfilm["index"].count()) # yes
print()

rentals_by_customer_left = rentals_by_customer_left.merge(film_category[["film_id","category_id"]],how = "left",\
                                               left_on = "film_id", right_on = "film_id")
rentals_by_customer_left

1000 958

True    16044
Name: film_id, dtype: int64

film_id
1    1000
Name: index, dtype: int64



,customer_id,inventory_id,film_id,title,category_id
0,1,797,174,CONFIDENTIAL INTERVIEW,12
1,1,726,159,CLOSER BANG,5
2,1,2639,579,MINDS TRUMAN,1
3,1,108,22,AMISTAD MIDSUMMER,13
4,1,2785,611,MUSKETEERS WAIT,4
...,...,...,...,...,...
16039,599,914,203,DAISY MENAGERIE,14
16040,599,2419,527,LOLA AGENT,11
16041,599,2033,443,HURRICANE AFFAIR,5
16042,599,4048,881,TEMPLE ATTRACTION,11


In [8]:
# join 4 - category name

# how many unique foreign keys are on both the rentals_by_customer table and the category table
print(category["category_id"].nunique(), rentals_by_customer_left["category_id"].nunique())
print()
# how many of the cat id's in the rentals_by_customer table exist on the  category table
print(rentals_by_customer_left.category_id.isin(category["category_id"]).value_counts())
print()
# all of them exist
# # are all cat ids on the category table unique?
groupcat = category["category_id"].value_counts().sort_values(ascending = False).reset_index().groupby("category_id")
print(groupcat["index"].count()) # yes
print()
rentals_by_customer_left = rentals_by_customer_left.merge(category[["category_id","name"]], how = "left", \
                                                          left_on = "category_id", right_on = "category_id")
rentals_by_customer_left


16 16

True    16044
Name: category_id, dtype: int64

category_id
1    16
Name: index, dtype: int64



,customer_id,inventory_id,film_id,title,category_id,name
0,1,797,174,CONFIDENTIAL INTERVIEW,12,Music
1,1,726,159,CLOSER BANG,5,Comedy
2,1,2639,579,MINDS TRUMAN,1,Action
3,1,108,22,AMISTAD MIDSUMMER,13,New
4,1,2785,611,MUSKETEERS WAIT,4,Classics
...,...,...,...,...,...,...
16039,599,914,203,DAISY MENAGERIE,14,Sci-Fi
16040,599,2419,527,LOLA AGENT,11,Horror
16041,599,2033,443,HURRICANE AFFAIR,5,Comedy
16042,599,4048,881,TEMPLE ATTRACTION,11,Horror


In [9]:
# final result - top 2 categories for each customer
rentals_by_customer = rentals[["customer_id","inventory_id"]].sort_values("customer_id")
rentals_by_customer = rentals_by_customer.merge(inventory[["inventory_id","film_id"]],\
                                                how = "left",left_on = "inventory_id", right_on ="inventory_id",)
rentals_by_customer = rentals_by_customer.merge(film[["film_id","title"]], how = "left", left_on = "film_id", right_on = \
                                               "film_id") 
rentals_by_customer = rentals_by_customer.merge(film_category[["film_id","category_id"]],how = "left",\
                                               left_on = "film_id", right_on = "film_id")
rentals_by_customer = rentals_by_customer.merge(category[["category_id","name"]], how = "left", left_on = "category_id", \
                                                right_on = "category_id")
rentals_by_customer = rentals_by_customer[["customer_id","title","name"]]
agg_rentals = rentals_by_customer.groupby(["customer_id","name"])["name"].count().reset_index(name='Count')
agg_rentals.sort_values(["customer_id","Count"], ascending = [True,False], inplace = True)
agg_rentals["Rank"] = agg_rentals.groupby("customer_id")["Count"].rank("first", ascending = False)
agg_rentals[agg_rentals["Rank"] <= 2].sort_values("customer_id", ascending = True)

,customer_id,name,Count,Rank
2,1,Classics,6,1.0
3,1,Comedy,5,2.0
25,2,Sports,5,1.0
17,2,Classics,4,2.0
27,3,Action,4,1.0
...,...,...,...,...
7705,597,Children,3,2.0
7721,598,Comedy,3,1.0
7723,598,Drama,3,2.0
7732,599,Comedy,3,1.0


In [10]:
# this one will be used for the comparison calculations and for information on favorite actors
Total_movies = rentals[["customer_id","inventory_id"]].sort_values("customer_id")
Total_movies = Total_movies.merge(inventory[["inventory_id","film_id"]],\
                                                how = "left",left_on = "inventory_id", right_on ="inventory_id",)
Total_movies = Total_movies.merge(film[["film_id","title"]], how = "left", left_on = "film_id", right_on = \
                                               "film_id") 
Total_movies = Total_movies.merge(film_category[["film_id","category_id"]],how = "left",\
                                               left_on = "film_id", right_on = "film_id")
Total_movies = Total_movies.merge(category[["category_id","name"]], how = "left", left_on = "category_id", \
                                                right_on = "category_id")

Total_movies = Total_movies[["customer_id","film_id","title","name"]]
Total_movies 

,customer_id,film_id,title,name
0,1,174,CONFIDENTIAL INTERVIEW,Music
1,1,159,CLOSER BANG,Comedy
2,1,579,MINDS TRUMAN,Action
3,1,22,AMISTAD MIDSUMMER,New
4,1,611,MUSKETEERS WAIT,Classics
...,...,...,...,...
16039,599,203,DAISY MENAGERIE,Sci-Fi
16040,599,527,LOLA AGENT,Horror
16041,599,443,HURRICANE AFFAIR,Comedy
16042,599,881,TEMPLE ATTRACTION,Horror
